# Self-Driving Car Engineer Nanodegree

## Computer Vision

## Project: Advanced Lane Finding

### Step 0: Declare and import dependencies on modules. Also declare global variables

In [5]:
#Declare dependencies on python modules here

In [6]:
#Declare global variables used here

### Step 1: Compute the camera calibration matrix and distortion coefficients given a set of chessboard images

### Step 2: Apply a distortion correction to raw images

### Step 3: Use color transforms, gradients, etc., to create a thresholded binary image

### Step 4: Apply a perspective transform to rectify binary image ("birds-eye view")

### Step 5: Detect lane pixels and fit to find the lane boundary

### Step 6: Determine the curvature of the lane and vehicle position with respect to center

### Step 7: Warp the detected lane boundaries back onto the original image

### Step 8: Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position

### Main module